In [ ]:
# Setup the Jupyter version of Dash
from jupyter_dash import JupyterDash

# Configure the necessary Python module imports for dashboard components
import dash_leaflet as dl
from dash import dcc, html, dash_table
from dash.exceptions import PreventUpdate
import plotly.express as px
from dash.dependencies import Input, Output, State
import base64

# Configure OS routines
import os

# Configure the plotting routines
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt



from animal_shelter import AnimalShelter

###########################
# Data Manipulation / Model
###########################

# Implementing Credentials & Python CRUD module info here.
username = "aacuser"
password = "animals"
shelter = AnimalShelter(
    username = username,
    password = password,
    host = "nv-desktop-services.apporto.com",
    port = 32327,
    db = "AAC",
    collection = "animals"
)


# Defining a tiny helper method here for get_df, which will hopefully help keep things a bit cleaner,
# and have the advantage of only needing to modify other things once (as opposed to multiple times throughout
# the file.)

def get_df(query = None):
    # Modifying the line below for the new Algorithm enhancement.
    records = shelter.read_cached(query or {})
    df = pd.DataFrame.from_records(records)
    if df.empty:
        return df
    
    # Dropping the ObjectID here, in order to avoid any DataTable type issues.
    df.drop(columns = ["_id"], inplace = True, errors = "ignore")
    return df

# Creating an initial, unfiltered view here.
df = get_df({})

# Creating filters here, requested by the client (<= 2 years old [= 104 weeks]).
RESCUE_QUERIES = {
    "All": {},
    "Water Rescue" : {
        "$and" : [
            {"animal_type" : "Dog"},
            {"breed" : {"$in" : ["Labrador Retriever Mix", "Chesapeake Bay Retriever", "Newfoundland"]}},
            {"age_upon_outcome_in_weeks" : {"$lte" : 104}},
        ]
    },

    "Mountain or Wilderness Rescue" : {
        "$and" : [
            {"animal_type" : "Dog"},
            {"breed" : {"$in" : ["German Shepherd", "Alaskan Malamute", "Old English Sheepdog", "Siberian Husky", "Rottweiler"]}},
            {"age_upon_outcome_in_weeks" : {"$lte" : 104}},
        ]
    },

    "Disaster or Individual Tracking" : {
        "$and" : [
            {"animal_type" : "Dog"},
            {"breed" : {"$in" : ["Doberman Pinscher", "German Shepherd", "Golden Retriever", "Bloodhound", "Rottweiler"]}},
            {"age_upon_outcome_in_weeks" : {"$lte" : 104}},
        ]
    },

}

#########################
# Dashboard Layout / View
#########################
app = JupyterDash(__name__)

# Implementing the Grazioso Salvare Logo here.
with open("Grazioso Salvare Logo.png", "rb") as f:
    encoded_logo = base64.b64encode(f.read()).decode()


app.layout = html.Div([
    # Creating the Header with both the logo and a unique identifier here.
    html.Div(style = {"display" : "flex", "alignItems" : "center", "gap" : "16px"},
             children = [
                 html.Img(src = f"data:image/png;base64,{encoded_logo}", style = {"height" : "80px"}),
                 html.Div(children = [
                     html.H1("Grazioso Salvare Dashboard", style = {"margin" : "0"}),
                     html.H4("SNHU CS-340 - David Deist", style = {"margin" : "0"}),
                 ])
             ]),
             html.Hr(),

             # Creating interactive filters here.
             html.Div([
                 dcc.RadioItems(
                     id = "filter-type",
                     options = [
                         {"label" : "All", "value" : "All"},
                         {"label" : "Water Rescue", "value" : "Water Rescue"},
                         {"label" : "Mountain / Wilderness Rescue", "value" : "Mountain or Wilderness Rescue"},
                         {"label" : "Disaster / Individual Tracking", "value" : "Disaster or Individual Tracking"},
                     ],
                     value = "All",
                     inline = True
                 )
             ]),

             # Implementing the Data Table here.
             dash_table.DataTable(
                 id = "datatable-id",
                 columns = [{"name" : i, "id" : i, "deletable" : False, "selectable" : True} for i in df.columns],
                 data = df.to_dict("records"),

                 row_selectable = "single",
                 selected_rows = [0],
                 page_action = "native",
                 page_size = 10,
                 sort_action = "native",
                 filter_action = "native",   # Using this for a quick text filter box in the Header.
                 style_table = {"overflowX" : "auto"},
                 style_cell = {"textAlign" : "left", "minWidth" : "120px", "width" : "120px", "maxWidth" : "220px", "whiteSpace" : "normal"},
                 
            ),

            html.Br(),
            html.Hr(),

            # Creating the map and the chart here, which will be side-by-side.
            html.Div(className = "row", style = {"display" : "flex", "gap" : "24px"}, children = [
                html.Div(id = "graph-id", className = "col s12 m6", style = {"flex" : "1"}),
                html.Div(id = "map-id", className = "col s12 m6", style = {"flex" : "1"}),
            ])
])




#############################################
# Interaction Between Components / Controller
#############################################

# Creating a filter here for the interactive data table with the MongoDB queries.
    
@app.callback(Output('datatable-id','data'),
              [Input('filter-type', 'value')])

def update_table(filter_value):
    query = RESCUE_QUERIES.get(filter_value, {})
    filtered = get_df(query)
    return filtered.to_dict("records")

# Here, the table will drive the chart.
@app.callback(
    Output('graph-id', 'children'),
    [Input('datatable-id', "derived_virtual_data")])

def update_chart(viewData):
    if viewData is None:
        raise PreventUpdate
    
    dff = pd.DataFrame.from_dict(viewData)

    if dff.empty:
        raise PreventUpdate
    
    fig = px.pie(dff, names = "breed", title = "Breeds in Current Selection")
    
    return [dcc.Graph(figure = fig)]


@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_columns')]
)
def update_styles(selected_columns):
    selected_columns = selected_columns or []
    return [{
        'if': { 'column_id': c },
        'background_color': '#D2F3FF'
    } for c in selected_columns]


# The table selection will drive the map here (lattitude & longitude via the column names).
@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_virtual_data"),
     Input('datatable-id', "derived_virtual_selected_rows")])
def update_map(viewData, selected_rows):  
    
    if viewData is None:
        raise PreventUpdate
    
    dff = pd.DataFrame.from_dict(viewData)

    if dff.empty:
        raise PreventUpdate
    
    row = selected_rows[0] if selected_rows else 0
    lat = dff.iloc[row].get("location_lat", None)
    lon = dff.iloc[row].get("location_long", None)
    breed = dff.iloc[row].get("breed", "Unknown")
    name = dff.iloc[row].get("name", "Unknown")

    if pd.isna(lat) or pd.isna(lon):
        raise PreventUpdate
        
    # Austin TX is at [30.75,-97.48]
    return [
        dl.Map(style={'width': '100%', 'height': '500px'}, center=[30.75,-97.48], zoom=10, children=[
            dl.TileLayer(id="base-layer-id"),
            
            # Marker with tool tip and popup
            dl.Marker(position=[lat, lon], children=[
                dl.Tooltip(str(breed)),
                dl.Popup([
                    html.H3("Animal Name"),
                    html.P(str(name))])
                ])
            ])
        ]



app.run_server(debug = True, mode = "external", port = 8050)
